In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

# ==========================================
# 1. OBTENCIÓN Y ESTUDIO DEL DATASET
# ==========================================
# Cargamos el dataset Breast Cancer Wisconsin
data = load_breast_cancer()

# Extraemos las características (X) y el rótulo (y)
X = data.data
y = data.target

# Convertimos a DataFrame para facilitar el análisis visual
df = pd.DataFrame(X, columns=data.feature_names)
df['target'] = y

print("--- INFORMACIÓN DEL DATASET ---")
print(f"Número de muestras: {df.shape[0]}")
print(f"Número de características (features): {df.shape[1] - 1}") # Restamos la columna target
print(f"Nombres de clases: {data.target_names}") # ['malignant' 'benign']
print("\n--- PRIMERAS 5 FILAS (VISTA PREVIA DE CARACTERÍSTICAS) ---")
print(df.head())

print("\n--- ESTADÍSTICAS DESCRIPTIVAS ---")
print(df.describe().transpose().head(10)) # Mostramos solo las primeras 10 para no saturar

# ==========================================
# 2. PREPROCESAMIENTO
# ==========================================
# Dividir en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ESTANDARIZACIÓN (CRÍTICO):
# Las redes neuronales requieren que las entradas tengan escalas similares.
# StandardScaler ajusta los datos para que tengan media 0 y varianza 1.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\nDimensiones de X_train (escalado): {X_train_scaled.shape}")

# ==========================================
# 3. DISEÑO DE LA RED NEURONAL
# ==========================================
model = tf.keras.Sequential([
    # Capa de Entrada implícita (30 neuronas, una por característica)

    # Capa Oculta 1: 16 neuronas, activación ReLU
    # Decisión de diseño: Suficiente capacidad para aprender patrones no lineales en 30 features.
    tf.keras.layers.Dense(16, activation='relu', input_shape=(30,)),

    # Capa Oculta 2: 8 neuronas, activación ReLU
    # Reduce la dimensionalidad progresivamente.
    tf.keras.layers.Dense(8, activation='relu'),

    # Capa de Salida: 1 neurona, activación Sigmoide
    # Razón: Es clasificación binaria (0 o 1). La sigmoide da una probabilidad entre 0 y 1.
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy', # Pérdida estándar para clasificación binaria
              metrics=['accuracy'])

model.summary()

# ==========================================
# 4. ENTRENAMIENTO
# ==========================================
print("\n--- INICIANDO ENTRENAMIENTO ---")
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=0)
print("Entrenamiento finalizado.")

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"\nPrecisión en el set de prueba: {accuracy*100:.2f}%")

# ==========================================
# 5. EJEMPLOS BASADOS EN PESOS APRENDIDOS
# ==========================================

# A. Predicciones con el modelo entrenado
# Tomamos 5 muestras del set de prueba para ver qué decide la red
print("\n--- EJEMPLOS DE PREDICCIÓN ---")
sample_indices = [0, 10, 20, 30, 40]
samples = X_test_scaled[sample_indices]
true_labels = y_test[sample_indices]

# La red usa sus pesos aprendidos para calcular esto:
predictions = model.predict(samples)

print(f"{'Probabilidad (Benigno)':<25} | {'Predicción':<12} | {'Realidad':<12}")
print("-" * 55)
for i, pred in enumerate(predictions):
    prob = pred[0]
    predicted_label = 1 if prob > 0.5 else 0
    clase_pred = data.target_names[predicted_label]
    clase_real = data.target_names[true_labels[i]]
    print(f"{prob:<25.4f} | {clase_pred:<12} | {clase_real:<12}")

# B. Inspección de los Pesos (Weights)
# Podemos ver qué características está ponderando más la primera capa.
print("\n--- ANÁLISIS DE PESOS (PRIMERA CAPA) ---")
weights, biases = model.layers[0].get_weights()
print(f"Forma de la matriz de pesos (Features x Neuronas): {weights.shape}")

# Calculamos la magnitud promedio de los pesos para cada característica de entrada
# Esto nos da una idea aproximada de la "importancia" que la red le dio a cada feature.
feature_importance = np.mean(np.abs(weights), axis=1)

# Creamos un DataFrame para visualizarlo mejor
importance_df = pd.DataFrame({
    'Característica': data.feature_names,
    'Peso_Promedio': feature_importance
}).sort_values(by='Peso_Promedio', ascending=False)

print("\nTop 5 Características más influyentes según los pesos aprendidos:")
print(importance_df.head(5))

# Visualización rápida de la pérdida durante el entrenamiento
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Pérdida (Train)')
plt.plot(history.history['val_loss'], label='Pérdida (Val)')
plt.title('Curva de Aprendizaje (Pérdida)')
plt.xlabel('Épocas')
plt.legend()
plt.show()